In [ ]:
import ee

ee.Authenticate()
ee.Initialize(project='ee-mwu336')

## Dataset

In [ ]:
start_year = 2019
end_year = 2023

## Generate CDL at MODIS scale

In [ ]:
MODIS=ee.ImageCollection('MODIS/061/MCD12Q2').filterDate(str(start_year)+'-01-01',str(end_year)+'-12-31')

# Get information about the MODIS projection and scale.
modisProjection = MODIS.first().projection()
print('MODIS projection:', modisProjection.getInfo())

scale = MODIS.first().projection().nominalScale()
print(scale.getInfo())

MODIS projection: {'type': 'Projection', 'crs': 'SR-ORG:6974', 'transform': [463.3127165279165, 0, -20015109.354, 0, -463.3127165279167, 10007554.677003]}
463.31271652791656


### US CDL

In [ ]:
import time

for year in range(start_year,end_year+1):

    ## export pure pixel of CDL in MODIS projection
    CDL_imgs = ee.ImageCollection('USDA/NASS/CDL')\
      .filterDate(str(year)+'-01-01',str(year)+'-12-31').select([0])
    CDL_img = ee.Image(CDL_imgs.first()).eq(1) # indicates corn

    CDL_img_mean = CDL_img.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
#       'bestEffort': True,
      'maxPixels': 1024
    }).reproject(**{
      'crs': modisProjection
    })

    region1 = ee.Geometry.Rectangle([-105, 35, -93, 49])
    region2 = ee.Geometry.Rectangle([-93, 35, -78, 49])

    while True:
        try:
            # export to assets
            task = ee.batch.Export.image.toAsset(**{
            'image': CDL_img_mean,
            'description': 'CDL_corn_1_'+str(year),
            'assetId': 'projects/ee-mwu336/assets/Corn_yield/CDL_500_MODIS/CDL_corn_1_'+str(year), # replace with your own path
            'scale': scale,
            'region': region1
            })
            task.start()

            time.sleep(10)

            task2 = ee.batch.Export.image.toAsset(**{
            'image': CDL_img_mean,
            'description': 'CDL_corn_2_'+str(year),
            'assetId': 'projects/ee-mwu336/assets/Corn_yield/CDL_500_MODIS/CDL_corn_2_'+str(year), # replace with your own path
            'scale': scale,
            'region': region2
            })
            task2.start()
        except:
            print('retry')
            time.sleep(60)
            continue
        break


## Compond Every CDL images

In [ ]:

for i in range(2019,2024):
    year=str(i)
    specific_date = ee.Date(year+'-01-01')
    CDL1 = ee.Image('projects/ee-mwu336/assets/Corn_yield/CDL_500_MODIS/CDL_corn_1_'+year)
    CDL2 = ee.Image('projects/ee-mwu336/assets/Corn_yield/CDL_500_MODIS/CDL_corn_2_'+year)
    CDL = ee.ImageCollection([CDL1, CDL2]).mosaic().set('system:time_start', specific_date.millis())

    region = ee.Geometry.Rectangle([-105, 35, -78, 49])

    task = ee.batch.Export.image.toAsset(**{
        'image': CDL,
        'description': 'CDL_corn_CB_'+str(year),
        'assetId': 'projects/ee-mwu336/assets/Corn_yield/CDL_500_MODIS/Compond/CDL_corn_CB_'+str(year), # replace with your own path
        'scale': scale,
        'region': region
        })
    try:
        task.start()
    except:
        print('fail')